In [1]:
import pandas as pd

df = pd.read_csv('Waste_Management_Facilities.csv')
# print(df.to_string()) 

In [2]:
df = df.loc[(df.state == "VIC")]
df = df.loc[(df.facility_infastructure_type == "E-WASTE DROP-OFF FACILITY") | (df.facility_infastructure_type == "E-WASTE RECYCLING FACILITY")]
df = df.filter(items=['X', 'Y', 'facility_management_type', 'facility_infastructure_type', 'facility_name', 'address', 'suburb', 'postcode'])
df = df.loc[(df.facility_name != "OFFICEWORKS") & (df.facility_name != "HARVEY NORMAN") & (df.facility_name != "THE GOOD GUYS")]
# df

In [3]:
print('check_NaN: ',df.isnull().sum().sum())

duplicateRows = df[df.duplicated(['facility_name'])]
print('check_duplicate_name: ',len(duplicateRows))

check_NaN:  0
check_duplicate_name:  0


In [5]:
df.to_csv('dispose_location.csv', index=False)

In [45]:
# %pip install pandas mysql-connector-python
# %pip install pandas mysql-connector-python paramiko


In [ ]:
import mysql.connector
import paramiko
from sshtunnel import SSHTunnelForwarder

# SSH连接配置
ssh_config = {
    "hostname": "54.252.18.185:22",
    "username": "ubuntu",
    "password": "your-aws-password"
}

# 建立SSH连接
ssh_client = paramiko.SSHClient()
ssh_client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
ssh_client.connect(**ssh_config)


# SSH隧道配置
tunnel_config = {
    "ssh_address_tuple": (ssh_config["hostname"], 22),
    "ssh_username": ssh_config["username"],
    "ssh_password": ssh_config["password"],
    "remote_bind_address": ("127.0.0.1", 3306)
}

# 建立SSH隧道
tunnel = SSHTunnelForwarder(**tunnel_config)
tunnel.start()


# MySQL连接配置
db_config = {
    "host": "127.0.0.1",  # 本地端口通过SSH隧道连接到远程MySQL
    "port": tunnel.local_bind_port,
    "user": "your-username",
    "password": "your-password",
    "database": "your-database-name"
}

# 连接到MySQL数据库
connection = mysql.connector.connect(**db_config)

# 将DataFrame写入数据库
df.to_sql(name='table_name', con=connection, if_exists='replace', index=False)

connection.close()
tunnel.stop()
ssh_client.close()
